In [564]:
import pandas as pd
import numpy as np
import re

# load data

In [567]:
pd_calls = pd.read_csv("Berkeley_PD_Calls_for_Service_2023.csv")
pd_calls.head()

,Incident_Number,CreateDatetime,Call_Type,Source,Progress,Priority,Dispositions,Block_Address,City,ZIP_Code,NonBerkeley_Address,ObjectId
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,911 Call,In Progress,Level 3,MDT Narrative Only,SHATTUCK AVE & CHANNING WAY,BERKELEY,94704.0,NaN,32827
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Telephone,In Progress,Level 2,MDT Narrative Only,2400 ASHBY AVE,BERKELEY,94705.0,NaN,32865
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Telephone,Cold Report,Level 4,MDT Narrative Only,700 PAGE ST,BERKELEY,94710.0,NaN,32871
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Telephone,Cold Report,Level 3,MDT Narrative Only,1800 SOLANO AVE,BERKELEY,94707.0,NaN,32877
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,911 Call,In Progress,Level 2,MDT Narrative Only,1300 DELAWARE ST,BERKELEY,94702.0,NaN,32889


In [568]:
alcohol = pd.read_csv("CA-ABC-LicenseReport.csv")
alcohol.head()

,License Number,Status,License Type,Orig. Iss. Date,Expir. Date,Primary Owner,Premises Addr.,Business Name,Geo Code
0,3048,ACTIVE,41,03/01/1976,08/31/2024,PAGNOL ET CIE INC,"1517 SHATTUCK AVE,BERKELEY, CA 94709Census Tr...",CHEZ PANISSE,103
1,3048,ACTIVE,58,03/01/1976,08/31/2024,PAGNOL ET CIE INC,"1517 SHATTUCK AVE,BERKELEY, CA 94709Census Tr...",CHEZ PANISSE,103
2,18492,ACTIVE,41,05/17/1978,02/28/2025,INTERNATIONAL HOUSE,"2299 PIEDMONT EXT,BERKELEY, CA 94720Census Tr...",INTERNATIONAL HOUSE,103
3,18492,ACTIVE,58,05/17/1978,02/28/2025,INTERNATIONAL HOUSE,"2299 PIEDMONT EXT,BERKELEY, CA 94720Census Tr...",INTERNATIONAL HOUSE,103
4,22040,ACTIVE,41,12/01/1974,10/31/2024,FATAPPLES INC,"1346 MARTIN LUTHER KING JR WY,BERKELEY, CA 94...",FATAPPLE'S RESTAURANT & BAKERY,103


In [569]:
business = pd.read_csv("Business_Licenses.csv")
business.head()

,APN,RecordID,BusDesc,B1_PER_SUB_TYPE,DBA,NAICS,Tax_Code,Employee_Num,Bus_Own_Type,B1_BUSINESS_NAME,...,B1_ADDRESS2,B1_CITY,B1_STATE,B1_ZIP,B1_CONTACT_TYPE,B1_FULL_ADDRESS,B1_SITUS_CITY,B1_SITUS_STATE,B1_SITUS_ZIP,Business_Location
0,064 423500501,BL-011399,CLOTHING STORE,Retail Trade,RADBIRD LLC,448140 - Family Clothing Stores,R,4,LLC,RADBIRD LLC,...,NaN,BERKELEY,CA,94705-2454,Business Owner,2938 DOMINGO AVE,BERKELEY,CA,94705.0,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583..."
1,058 219101400,BL-049043,CATERING,Retail Trade,LA CREPE A MOI,722320 - Caterers,R,7,Sole Ownership,LA CREPE A MOI,...,NaN,BERKELEY,CA,94709-1318,Business Owner,2432 NINTH ST,BERKELEY,CA,94710.0,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980..."
2,NaN,BL-002916,LOCKSMITH,Business Personal Repair Svs,GLENVIEW KEY & LOCK,561622 - Locksmiths,B,8,Corporation,GLENVIEW KEY & LOCK,...,NaN,OAKLAND,CA,94602-1206,Business Owner,0 VARIOUS,BERKELEY,CA,94704.0,"0 VARIOUS\nBERKELEY, CA 94704\n(37.8682724, -1..."
3,NaN,BL-014178,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,NaN,CC,8,LLC,THE APOTHECARIUM,...,NaN,SANTA ROSA,CA,95404,Business Owner,2312 TELEGRAPH AVE,BERKELEY,CA,94704.0,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86..."
4,055 183200701,BL-043653,MASSAGE THERAPY,Massage,PAPEN GINA,812199 - Other Personal Care Services,BM,0,Sole Ownership,NaN,...,NaN,BERKELEY,CA,94704-2841,Business Owner,2337 PARKER ST 12,BERKELEY,CA,94704.0,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863..."


# make address seperate

In [574]:
# pd_calls make address intersection / # / street
pd_calls.head()

,Incident_Number,CreateDatetime,Call_Type,Source,Progress,Priority,Dispositions,Block_Address,City,ZIP_Code,NonBerkeley_Address,ObjectId
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,911 Call,In Progress,Level 3,MDT Narrative Only,SHATTUCK AVE & CHANNING WAY,BERKELEY,94704.0,NaN,32827
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Telephone,In Progress,Level 2,MDT Narrative Only,2400 ASHBY AVE,BERKELEY,94705.0,NaN,32865
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Telephone,Cold Report,Level 4,MDT Narrative Only,700 PAGE ST,BERKELEY,94710.0,NaN,32871
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Telephone,Cold Report,Level 3,MDT Narrative Only,1800 SOLANO AVE,BERKELEY,94707.0,NaN,32877
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,911 Call,In Progress,Level 2,MDT Narrative Only,1300 DELAWARE ST,BERKELEY,94702.0,NaN,32889


In [620]:
pd_calls['intersection'] = pd_calls['Block_Address'].apply(lambda x: '&' in x)
pd_calls.head()

,Incident_Number,CreateDatetime,Call_Type,Source,Progress,Priority,Dispositions,Block_Address,City,ZIP_Code,NonBerkeley_Address,ObjectId,intersection
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,911 Call,In Progress,Level 3,MDT Narrative Only,SHATTUCK AVE & CHANNING WAY,BERKELEY,94704.0,NaN,32827,True
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Telephone,In Progress,Level 2,MDT Narrative Only,2400 ASHBY AVE,BERKELEY,94705.0,NaN,32865,False
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Telephone,Cold Report,Level 4,MDT Narrative Only,700 PAGE ST,BERKELEY,94710.0,NaN,32871,False
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Telephone,Cold Report,Level 3,MDT Narrative Only,1800 SOLANO AVE,BERKELEY,94707.0,NaN,32877,False
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,911 Call,In Progress,Level 2,MDT Narrative Only,1300 DELAWARE ST,BERKELEY,94702.0,NaN,32889,False


In [622]:
pd_calls.loc[~pd_calls['intersection'], 'add_num'] = pd_calls.loc[~pd_calls['intersection'], 'Block_Address'].apply(lambda x: x.split()[0])
pd_calls.loc[~pd_calls['intersection'], 'street'] = pd_calls.loc[~pd_calls['intersection'], 'Block_Address'].apply(lambda x: ' '.join(x.split()[1:]))

In [624]:
#intersection true -> add_num & street = 0
pd_calls.loc[pd_calls['intersection'], ['add_num', 'street']] = 0

In [628]:
pd_calls = pd_calls[['Incident_Number', 'CreateDatetime', 'Call_Type', 'Priority', 'Block_Address', 'intersection', 'add_num', 'street']]

In [630]:
pd_calls.head()

,Incident_Number,CreateDatetime,Call_Type,Priority,Block_Address,intersection,add_num,street
0,2023-00062204,2023/12/29 20:27:46+00,CFS 1182 - Non Injury,Level 3,SHATTUCK AVE & CHANNING WAY,True,0,0
1,2023-00062206,2023/12/29 20:26:14+00,CFS 1042 - Welfare Check,Level 2,2400 ASHBY AVE,False,2400,ASHBY AVE
2,2023-00062200,2023/12/29 20:12:38+00,CFS FOUND - Found Property,Level 4,700 PAGE ST,False,700,PAGE ST
3,2023-00062228,2023/12/29 20:05:59+00,CFS 20002 - Hit & Run Prop.,Level 3,1800 SOLANO AVE,False,1800,SOLANO AVE
4,2023-00062202,2023/12/29 19:45:10+00,CFS 415 - Disturbance,Level 2,1300 DELAWARE ST,False,1300,DELAWARE ST


In [634]:
business.head()

,BusDesc,B1_PER_SUB_TYPE,DBA,Business_Location,add,add_num,street,new_name,alcohol
0,CLOTHING STORE,Retail Trade,RADBIRD LLC,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE,2938,DOMINGOAVE,RADBIRDLLC,False
1,CATERING,Retail Trade,LA CREPE A MOI,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST,2432,NINTHST,LACREPEAMOI,False
3,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE,2312,TELEGRAPHAVE,THEAPOTHECARIUM,False
4,MASSAGE THERAPY,Massage,PAPEN GINA,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12,2337,PARKERST,PAPENGINA,False
6,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE,2210,SANPABLOAVE,AVSSPECIALISTS,False


In [632]:
pd_calls.to_csv('prepared_pd_calls.csv')

In [576]:
# business make address intersection / # / street
business['add'] = business['Business_Location'].str.split('\n').str[0]
business.head()

,APN,RecordID,BusDesc,B1_PER_SUB_TYPE,DBA,NAICS,Tax_Code,Employee_Num,Bus_Own_Type,B1_BUSINESS_NAME,...,B1_CITY,B1_STATE,B1_ZIP,B1_CONTACT_TYPE,B1_FULL_ADDRESS,B1_SITUS_CITY,B1_SITUS_STATE,B1_SITUS_ZIP,Business_Location,add
0,064 423500501,BL-011399,CLOTHING STORE,Retail Trade,RADBIRD LLC,448140 - Family Clothing Stores,R,4,LLC,RADBIRD LLC,...,BERKELEY,CA,94705-2454,Business Owner,2938 DOMINGO AVE,BERKELEY,CA,94705.0,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE
1,058 219101400,BL-049043,CATERING,Retail Trade,LA CREPE A MOI,722320 - Caterers,R,7,Sole Ownership,LA CREPE A MOI,...,BERKELEY,CA,94709-1318,Business Owner,2432 NINTH ST,BERKELEY,CA,94710.0,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST
2,NaN,BL-002916,LOCKSMITH,Business Personal Repair Svs,GLENVIEW KEY & LOCK,561622 - Locksmiths,B,8,Corporation,GLENVIEW KEY & LOCK,...,OAKLAND,CA,94602-1206,Business Owner,0 VARIOUS,BERKELEY,CA,94704.0,"0 VARIOUS\nBERKELEY, CA 94704\n(37.8682724, -1...",0 VARIOUS
3,NaN,BL-014178,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,NaN,CC,8,LLC,THE APOTHECARIUM,...,SANTA ROSA,CA,95404,Business Owner,2312 TELEGRAPH AVE,BERKELEY,CA,94704.0,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE
4,055 183200701,BL-043653,MASSAGE THERAPY,Massage,PAPEN GINA,812199 - Other Personal Care Services,BM,0,Sole Ownership,NaN,...,BERKELEY,CA,94704-2841,Business Owner,2337 PARKER ST 12,BERKELEY,CA,94704.0,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12


In [578]:
 business[business['add'] == '0 VARIOUS'].shape[0]

4795

In [580]:
business = business[business['add'] != '0 VARIOUS']

In [582]:
business.head()

,APN,RecordID,BusDesc,B1_PER_SUB_TYPE,DBA,NAICS,Tax_Code,Employee_Num,Bus_Own_Type,B1_BUSINESS_NAME,...,B1_CITY,B1_STATE,B1_ZIP,B1_CONTACT_TYPE,B1_FULL_ADDRESS,B1_SITUS_CITY,B1_SITUS_STATE,B1_SITUS_ZIP,Business_Location,add
0,064 423500501,BL-011399,CLOTHING STORE,Retail Trade,RADBIRD LLC,448140 - Family Clothing Stores,R,4,LLC,RADBIRD LLC,...,BERKELEY,CA,94705-2454,Business Owner,2938 DOMINGO AVE,BERKELEY,CA,94705.0,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE
1,058 219101400,BL-049043,CATERING,Retail Trade,LA CREPE A MOI,722320 - Caterers,R,7,Sole Ownership,LA CREPE A MOI,...,BERKELEY,CA,94709-1318,Business Owner,2432 NINTH ST,BERKELEY,CA,94710.0,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST
3,NaN,BL-014178,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,NaN,CC,8,LLC,THE APOTHECARIUM,...,SANTA ROSA,CA,95404,Business Owner,2312 TELEGRAPH AVE,BERKELEY,CA,94704.0,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE
4,055 183200701,BL-043653,MASSAGE THERAPY,Massage,PAPEN GINA,812199 - Other Personal Care Services,BM,0,Sole Ownership,NaN,...,BERKELEY,CA,94704-2841,Business Owner,2337 PARKER ST 12,BERKELEY,CA,94704.0,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12
6,NaN,BL-040875,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,811111 - General Automotive Repair,B,3,Corporation,AVS SPECIALISTS,...,BERKELEY,CA,94710,Business Owner,2210 SAN PABLO AVE,BERKELEY,CA,94702.0,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE


In [584]:
business['add_num'] = business['add'].str.split().str[0]
business['add_num'] = business['add_num'].str.replace(' ', '')

business['street'] = business['add'].str.replace(r'\d+', '', regex=True)
business['street'] = business['street'].str.replace(' ', '')

business.head()

,APN,RecordID,BusDesc,B1_PER_SUB_TYPE,DBA,NAICS,Tax_Code,Employee_Num,Bus_Own_Type,B1_BUSINESS_NAME,...,B1_ZIP,B1_CONTACT_TYPE,B1_FULL_ADDRESS,B1_SITUS_CITY,B1_SITUS_STATE,B1_SITUS_ZIP,Business_Location,add,add_num,street
0,064 423500501,BL-011399,CLOTHING STORE,Retail Trade,RADBIRD LLC,448140 - Family Clothing Stores,R,4,LLC,RADBIRD LLC,...,94705-2454,Business Owner,2938 DOMINGO AVE,BERKELEY,CA,94705.0,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE,2938,DOMINGOAVE
1,058 219101400,BL-049043,CATERING,Retail Trade,LA CREPE A MOI,722320 - Caterers,R,7,Sole Ownership,LA CREPE A MOI,...,94709-1318,Business Owner,2432 NINTH ST,BERKELEY,CA,94710.0,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST,2432,NINTHST
3,NaN,BL-014178,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,NaN,CC,8,LLC,THE APOTHECARIUM,...,95404,Business Owner,2312 TELEGRAPH AVE,BERKELEY,CA,94704.0,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE,2312,TELEGRAPHAVE
4,055 183200701,BL-043653,MASSAGE THERAPY,Massage,PAPEN GINA,812199 - Other Personal Care Services,BM,0,Sole Ownership,NaN,...,94704-2841,Business Owner,2337 PARKER ST 12,BERKELEY,CA,94704.0,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12,2337,PARKERST
6,NaN,BL-040875,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,811111 - General Automotive Repair,B,3,Corporation,AVS SPECIALISTS,...,94710,Business Owner,2210 SAN PABLO AVE,BERKELEY,CA,94702.0,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE,2210,SANPABLOAVE


# merge business & alcohol

In [587]:
# business DBA
business['DBA']

0                        RADBIRD LLC
1                     LA CREPE A MOI
3                   THE APOTHECARIUM
4                         PAPEN GINA
6                    AVS SPECIALISTS
                    ...             
13336    PIADINA ORIGINAL MARINA LLC
13337                     FURAXA INC
13339                   LIAO CHI JUN
13340    LOVEJUST COMMUNITY WELLNESS
13343     WANG JOHN HELEN & CLIFFORD
Name: DBA, Length: 8549, dtype: object

In [589]:
business['new_name'] = business['DBA'].str.replace(" ","",regex = False)
business['new_name']

0                       RADBIRDLLC
1                      LACREPEAMOI
3                  THEAPOTHECARIUM
4                        PAPENGINA
6                   AVSSPECIALISTS
                   ...            
13336     PIADINAORIGINALMARINALLC
13337                    FURAXAINC
13339                   LIAOCHIJUN
13340    LOVEJUSTCOMMUNITYWELLNESS
13343       WANGJOHNHELEN&CLIFFORD
Name: new_name, Length: 8549, dtype: object

In [591]:
alcohol['new_name'] = alcohol['Business Name'].str.replace(" ","",regex = False)
alcohol['new_name']

0                      CHEZPANISSE
1                      CHEZPANISSE
2               INTERNATIONALHOUSE
3               INTERNATIONALHOUSE
4      FATAPPLE'SRESTAURANT&BAKERY
                  ...             
418    GREEKTHEATREATUCBERKELEYTHE
419                 FOODTOWNMARKET
420                    PHOSHODOUGH
421                BUTTERFISHSUSHI
422               VICTORYPOINTCAFE
Name: new_name, Length: 423, dtype: object

In [593]:
alcohol['new_owner'] = alcohol['Primary Owner'].str.replace(" ","",regex = False)
alcohol['new_owner']

0               PAGNOLETCIEINC
1               PAGNOLETCIEINC
2           INTERNATIONALHOUSE
3           INTERNATIONALHOUSE
4                 FATAPPLESINC
                ...           
418               DLSEVENTSLLC
419                   ALGUMAIE
420           PHOSHODOUGH,INC.
421           MCGEE,ANDREWJOHN
422    GURUKRUPAENTERPRISESLLC
Name: new_owner, Length: 423, dtype: object

In [595]:
alcohol['new_name'] = alcohol['new_name'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
alcohol['new_owner'] = alcohol['new_owner'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
business['new_name'] = business['new_name'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

In [597]:
alcohol.loc[:, 'new_owner'] = alcohol['new_owner'].str.replace(r'INC', '', regex=True).str.strip()
alcohol.loc[:, 'new_name'] = alcohol['new_name'].str.replace(r'INC', '', regex=True).str.strip()

business.loc[:, 'new_name'] = business['new_name'].str.replace(r'INC', '', regex=True).str.strip()

In [599]:
alcohol = alcohol[alcohol['Status'] == 'ACTIVE']
alcohol = alcohol.drop_duplicates(subset='new_name', keep='first').reset_index()
alcohol = alcohol.drop(columns='index')
alcohol

,License Number,Status,License Type,Orig. Iss. Date,Expir. Date,Primary Owner,Premises Addr.,Business Name,Geo Code,new_name,new_owner
0,3048,ACTIVE,41,03/01/1976,08/31/2024,PAGNOL ET CIE INC,"1517 SHATTUCK AVE,BERKELEY, CA 94709Census Tr...",CHEZ PANISSE,103,CHEZPANISSE,PAGNOLETCIE
1,18492,ACTIVE,41,05/17/1978,02/28/2025,INTERNATIONAL HOUSE,"2299 PIEDMONT EXT,BERKELEY, CA 94720Census Tr...",INTERNATIONAL HOUSE,103,INTERNATIONALHOUSE,INTERNATIONALHOUSE
2,22040,ACTIVE,41,12/01/1974,10/31/2024,FATAPPLES INC,"1346 MARTIN LUTHER KING JR WY,BERKELEY, CA 94...",FATAPPLE'S RESTAURANT & BAKERY,103,FATAPPLESRESTAURANTBAKERY,FATAPPLES
3,32395,ACTIVE,51,03/02/1959,05/31/2025,BAY AREA LODGE # 70 I.B.P.O.E. OF W.,"2835 SAN PABLO AVE,BERKELEY, CA 94702-2241Cen...",PEERLESS LODGE 1146,103,PEERLESSLODGE1146,BAYAREALODGE70IBPOEOFW
4,36781,ACTIVE,58,04/16/1965,01/31/2025,BERKELEY YACHT CLUB,"1 SEAWALL DR,BERKELEY, CA 94710Census Tract: ...",BERKELEY YACHT CLUB,103,BERKELEYYACHTCLUB,BERKELEYYACHTCLUB
...,...,...,...,...,...,...,...,...,...,...,...
337,657743,ACTIVE,47,06/05/2024,05/31/2025,DLS EVENTS LLC,HEARST GREEK THEATRE GAYLEY RD & STADIUM RIM W...,GREEK THEATRE AT UC BERKELEY THE,103,GREEKTHEATREATUCBERKELEYTHE,DLSEVENTSLLC
338,657771,ACTIVE,20,07/08/2024,06/30/2025,AL GUMAIE,"3049 SACRAMENTO ST,BERKELEY, CA 94702-2734Cen...",FOOD TOWN MARKET,103,FOODTOWNMARKET,ALGUMAIE
339,657820,ACTIVE,41,06/10/2024,05/31/2025,"PHOSHO DOUGH, INC.","1235 SAN PABLO AVE,BERKELEY, CA 94706-2217Cen...",PHOSHO DOUGH,103,PHOSHODOUGH,PHOSHODOUGH
340,657851,ACTIVE,41,05/14/2024,04/30/2025,"MCGEE, ANDREW JOHN","915 UNIVERSITY AVE,BERKELEY, CA 94710Census T...",BUTTERFISH SUSHI,103,BUTTERFISHSUSHI,MCGEEANDREWJOHN


In [601]:
alcohol = alcohol.dropna(subset=['Business Name'])

In [603]:
alcohol = alcohol.copy()
alcohol.loc[:, 'add'] = alcohol['Premises Addr.'].str.split(',').str[0]
alcohol.loc[:, 'add_num'] = alcohol['add'].str.split().str[0]
alcohol.loc[:, 'add_num'] = alcohol['add_num'].str.replace(' ', '')

alcohol.head()

,License Number,Status,License Type,Orig. Iss. Date,Expir. Date,Primary Owner,Premises Addr.,Business Name,Geo Code,new_name,new_owner,add,add_num
0,3048,ACTIVE,41,03/01/1976,08/31/2024,PAGNOL ET CIE INC,"1517 SHATTUCK AVE,BERKELEY, CA 94709Census Tr...",CHEZ PANISSE,103,CHEZPANISSE,PAGNOLETCIE,1517 SHATTUCK AVE,1517
1,18492,ACTIVE,41,05/17/1978,02/28/2025,INTERNATIONAL HOUSE,"2299 PIEDMONT EXT,BERKELEY, CA 94720Census Tr...",INTERNATIONAL HOUSE,103,INTERNATIONALHOUSE,INTERNATIONALHOUSE,2299 PIEDMONT EXT,2299
2,22040,ACTIVE,41,12/01/1974,10/31/2024,FATAPPLES INC,"1346 MARTIN LUTHER KING JR WY,BERKELEY, CA 94...",FATAPPLE'S RESTAURANT & BAKERY,103,FATAPPLESRESTAURANTBAKERY,FATAPPLES,1346 MARTIN LUTHER KING JR WY,1346
3,32395,ACTIVE,51,03/02/1959,05/31/2025,BAY AREA LODGE # 70 I.B.P.O.E. OF W.,"2835 SAN PABLO AVE,BERKELEY, CA 94702-2241Cen...",PEERLESS LODGE 1146,103,PEERLESSLODGE1146,BAYAREALODGE70IBPOEOFW,2835 SAN PABLO AVE,2835
4,36781,ACTIVE,58,04/16/1965,01/31/2025,BERKELEY YACHT CLUB,"1 SEAWALL DR,BERKELEY, CA 94710Census Tract: ...",BERKELEY YACHT CLUB,103,BERKELEYYACHTCLUB,BERKELEYYACHTCLUB,1 SEAWALL DR,1


In [605]:
mapping = {
    'MHMARKETLIQUOR': 'MHMARKET',
    'KIRALA': 'KIRALARESTAURANT',
    'CACTUSTAQUERIA': 'CACTUSTAQUERIASOLANO',
    'ETHIOPIARESTAURANT': 'MESKIESETHIOPIARESTAURANT',
    'ASHKENAZMUSICDANCECOMMUNITYCENTER': 'ASHKENAZMUSICDANCECOMMCT',
    'BERKELEYBOWLPRODUCE': 'BERKELEYBOWLMARKET',
    'SAULSRESTAURANTDELICATESSEN': 'SAULSRESTAURANTDELIGROCERY',
    'CHIPOTLEMEXICANGRILL': 'CHIPOTLEMEXICANGRILL231',
    'TRUMERBREWERY': 'TRUMERBRAUEREI',
    '7ELEVEN223217582C': '7ELEVEN17582CGREENCOLOURS',
    'SEABREEZEMARKETDELI': 'SEABREEZEMARKET',
    'SHATTUCKHOTELPLAZA': 'SHATTUCKHOTEL',
    'SHOTGUNPLAYERS': 'THESHOTGUNPLAYERS',
    'AFIKOMENJUDAICA': 'AFIKOMEN',
    'LEBATEAUIVRETHEDRUNKENBOAT': 'LEBATEAUIVREDRUNKENBOAT',
    'IPPUKURESTAURANT': 'IPPUKU',
    'MANPUKU': 'MANPUKURESTAURANT',
    'CVSPHARMACY10121': 'GARFIELDBEACHCVSLLC10121',
    'MINTLEAFVIETNAMESERESTAURANT': 'MINTLEAFVIETNAMESEREST',
    'SPOONKOREANBISTRO': 'SPOON',
    'TUPPERREED': 'TUPPERANDREED',
    'GREATCHINARESTAURANT': 'THEGREATCHINA',
    'KAMADOSUSHI': 'JJFOODSPECIALTIESDBAKAMADOSUSHI',
    'FRANKLINBROSMARKETLLC': 'FRANKLINBROSMARKET',
    'MOUNTEVERESTRESTAURANT': 'PASUPATINATHDBAMOUNTEVERESTRESTAURANT',
    'LAMISSIONMEXICANGRILL': 'LAMISSION',
    'CASALATINABAKERY': 'CASALATINA',
    'NABOLOMBAKERYPIZZERIA': 'NOBOLOMBAKERY',
    'LEPHO': 'PHOVIETNAMESERESTAURANT',
    'STANFORDLIQUORS': 'STANFORDLIQUOR',
    'ANDRONICOSCOMMUNITYMARKET2451': 'SAFEWAYCOMMUNITYMARKETS2451',
    'ANDRONICOSCOMMUNITYMARKET2453': 'SAFEWAYCOMMUNITYMARKETS2453',
    'CHIPOTLEMEXICANGRILLSTORE2703-': 'CHIPOTLEMEXICANGRILL2303',
    'AKIS': 'AKIJAPANESERESTAURANT',
    'GRADUATEBERKELEY': 'GRADUATEBERKELEYHOTEL',
    'SICHUANSTYLERESTAURANT': 'SICHUANSTYLE',
    'TARGETT3267': 'TARTETSTORET3267',
    'BUTCHERSSONTHE': 'THEBURCHERSSON',
    'NEWCEDARMARKET': 'CEDARMARKET',
    'SPATSSHATTUCKAVENUESPATS': '1974SHATTUCKAVENUELLC',
    'LEMATETHIOPIANRESTAURANTANDCAFELLC': 'LEMATETHIOPIANRESTAURANTANDCAFE',
    'WINENOIRELLCTHE': 'THEWINENOIRELLC',
    'HANAJAPAN': 'HANAJAPANSTEAKHOUSE',
    'TAPHAUSTHE': 'THETAPHAUS',
    'VAULTCAFETHE': 'THEVAULTCAFE',
    'BERKELEYBOWLMARKETPLACE': 'BERKELEYBOWLWEST',
    'DOUBLETREEBYHILTONBERKELYMARINA': 'DOUBLETREEBYHILTONHOTELBERKELYMARINA',
    'FLAVORSOFINDIA': 'NEWFLAVORSOFINDIA',
    'KOKOLO': 'KOKOLODONBURI',
    'NATURALGROCERYCOMPANYBERKELEYNATURALGROCERYCOTHE': 'BERKELEYNATURALGROCERY',
    'ROSESONADELINE': 'ROSESONADELINELLC',
    'CLCONVENIENVESTORE': 'CLCONVIENCESTORE',
    'JAZZCAFFETHE': 'THEJAZZCAFFE',
    'LOCOSTLIQUORS': 'LOCOSTLIQUOR',
    'VENUSSOLANO': 'VENUS',
    'BOPSHOP': 'BOPSHOPKOREANKITCHEN',
    'THAIDELIGHTCUISINE': 'THAIDELIGHT',
    'DELIRAMAAPYROSPASTRAMICO': 'DELIRAMA',
    'QUEENOFSHEBA2': 'QUEENOFSHEBAMARKET',
    'SPANISHTABLETHE': 'THESPANISHTABLE',
    'USGASFOOD': 'UCGASFOODMARKT',
    'CELLARMAKERBREWINGCO': 'CELLARMAKERBREWINGCOMPANY',
    'WALNUTTAVERNTHE': 'WALNUTSTC',
    'JAMTHAIRESTAURANT': 'THASIAMTHAIBISTRO',
    'PHOTASTY': 'PHOTASTYLLC',
    'BRAZILCAFEWESTBRAEBEIRGARTEN': 'BRAZILCAFEATWESTBRAEBIERGARTEN',
    'BUTTERFISHSUSHI': 'BUTTERFISHSUSHILLC'
}

alcohol['new_name'] = alcohol['new_name'].replace(mapping)

In [607]:
business_names = set(business['new_name'])
alcohol.loc[:, 'in'] = alcohol.apply(lambda row: row['new_name'] in business_names or row['new_owner'] in business_names, axis=1)

In [609]:
alcohol_names = set(alcohol['new_name'])
alcohol_owners = set(alcohol['new_owner'])

business['alcohol'] = business['new_name'].apply(
    lambda name: name in alcohol_names or name in alcohol_owners
)
business.head()

,APN,RecordID,BusDesc,B1_PER_SUB_TYPE,DBA,NAICS,Tax_Code,Employee_Num,Bus_Own_Type,B1_BUSINESS_NAME,...,B1_FULL_ADDRESS,B1_SITUS_CITY,B1_SITUS_STATE,B1_SITUS_ZIP,Business_Location,add,add_num,street,new_name,alcohol
0,064 423500501,BL-011399,CLOTHING STORE,Retail Trade,RADBIRD LLC,448140 - Family Clothing Stores,R,4,LLC,RADBIRD LLC,...,2938 DOMINGO AVE,BERKELEY,CA,94705.0,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE,2938,DOMINGOAVE,RADBIRDLLC,False
1,058 219101400,BL-049043,CATERING,Retail Trade,LA CREPE A MOI,722320 - Caterers,R,7,Sole Ownership,LA CREPE A MOI,...,2432 NINTH ST,BERKELEY,CA,94710.0,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST,2432,NINTHST,LACREPEAMOI,False
3,NaN,BL-014178,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,NaN,CC,8,LLC,THE APOTHECARIUM,...,2312 TELEGRAPH AVE,BERKELEY,CA,94704.0,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE,2312,TELEGRAPHAVE,THEAPOTHECARIUM,False
4,055 183200701,BL-043653,MASSAGE THERAPY,Massage,PAPEN GINA,812199 - Other Personal Care Services,BM,0,Sole Ownership,NaN,...,2337 PARKER ST 12,BERKELEY,CA,94704.0,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12,2337,PARKERST,PAPENGINA,False
6,NaN,BL-040875,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,811111 - General Automotive Repair,B,3,Corporation,AVS SPECIALISTS,...,2210 SAN PABLO AVE,BERKELEY,CA,94702.0,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE,2210,SANPABLOAVE,AVSSPECIALISTS,False


In [611]:
business.isna().sum()

APN                   581
RecordID                0
BusDesc                 3
B1_PER_SUB_TYPE         0
DBA                     0
NAICS                  14
Tax_Code                1
Employee_Num            0
Bus_Own_Type          110
B1_BUSINESS_NAME      400
B1_ADDRESS1             0
B1_ADDRESS2          8515
B1_CITY                 0
B1_STATE                4
B1_ZIP                  0
B1_CONTACT_TYPE         0
B1_FULL_ADDRESS        32
B1_SITUS_CITY           8
B1_SITUS_STATE          9
B1_SITUS_ZIP           10
Business_Location       0
add                     0
add_num                 0
street                  0
new_name                0
alcohol                 0
dtype: int64

In [613]:
columns_to_keep = [
    'BusDesc', 'B1_PER_SUB_TYPE', 'DBA',
    'Business_Location', 'add', 'add_num', 'street', 
    'new_name', 'alcohol'
]

business = business[columns_to_keep]

business.head()

,BusDesc,B1_PER_SUB_TYPE,DBA,Business_Location,add,add_num,street,new_name,alcohol
0,CLOTHING STORE,Retail Trade,RADBIRD LLC,"2938 DOMINGO AVE\nBERKELEY, CA 94705\n(37.8583...",2938 DOMINGO AVE,2938,DOMINGOAVE,RADBIRDLLC,False
1,CATERING,Retail Trade,LA CREPE A MOI,"2432 NINTH ST\nBERKELEY, CA 94710\n(37.8611980...",2432 NINTH ST,2432,NINTHST,LACREPEAMOI,False
3,CANNABIS DISPENSARY - MEDICAL,Cannabis,THE APOTHECARIUM,"2312 TELEGRAPH AVE\nBERKELEY, CA 94704\n(37.86...",2312 TELEGRAPH AVE,2312,TELEGRAPHAVE,THEAPOTHECARIUM,False
4,MASSAGE THERAPY,Massage,PAPEN GINA,"2337 PARKER ST 12\nBERKELEY, CA 94704\n(37.863...",2337 PARKER ST 12,2337,PARKERST,PAPENGINA,False
6,AUTOMOTIVE REPAIR,Business Personal Repair Svs,AVS SPECIALISTS,"2210 SAN PABLO AVE\nBERKELEY, CA 94702\n(37.86...",2210 SAN PABLO AVE,2210,SANPABLOAVE,AVSSPECIALISTS,False


In [615]:
business.to_csv('business+alcohol.csv')